In [13]:
pip install tmdbsimple


You should consider upgrading via the '/Users/beebo/.pyenv/versions/3.8.12/envs/objectively_funny/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [1]:
import pandas as pd

# Preparation

In [2]:
df = pd.read_json('/Users/beebo/code/rmelbardis/ObjectivelyFunny/raw_data/filtered_scraps_from_the_loft.json')
df.head()

,url,title,full_transcript
0,https://scrapsfromtheloft.com/comedy/jim-gaffi...,Jim Gaffigan: Comedy Monster (2021) | Transcript,"Thank you! Thank you! Oh, my gosh. Thank you s..."
1,https://scrapsfromtheloft.com/comedy/louis-c-k...,Louis C. K.: Sorry (2021) | Transcript,♪♪ [“Like a Rolling Stone” by Bob Dylan playin...
2,https://scrapsfromtheloft.com/comedy/drew-mich...,Drew Michael: Red Blue Green (2021) | Transcript,(EMOTIONAL MUSIC PLAYING)\n\n(MUSIC ENDS)\n\nD...
3,https://scrapsfromtheloft.com/comedy/mo-amer-m...,Mo Amer: Mohammed in Texas (2021) | Transcript,[quirky flute music playing]\n\n[single note p...
4,https://scrapsfromtheloft.com/comedy/dave-chap...,Dave Chappelle: The Closer (2021) | Transcript,[audience murmuring]\n\n[murmuring continues]\...


In [3]:
title = df['title']

In [4]:
df.sort_values('title')[:10]

,url,title,full_transcript
195,https://scrapsfromtheloft.com/comedy/adel-kara...,ADEL KARAM: LIVE FROM BEIRUT (2018) – Full Tra...,A NETFLIX COMEDY SPECIAL\nRecorded at the Casi...
276,https://scrapsfromtheloft.com/comedy/al-madrig...,AL MADRIGAL: WHY IS THE RABBIT CRYING? (2013) ...,[dog barks] [FisherGreen’s Sisters Brothers pl...
281,https://scrapsfromtheloft.com/comedy/ali-wong-...,ALI WONG: BABY COBRA (2016) – Full Transcript,"Ladies and gentlemen, please welcome to the st..."
291,https://scrapsfromtheloft.com/comedy/amy-schum...,AMY SCHUMER: LIVE AT THE APOLLO (2015) – Full ...,"My name is Amy, and it’s my show! ♪ Man ♪ ♪ Uh..."
289,https://scrapsfromtheloft.com/comedy/amy-schum...,AMY SCHUMER: MOSTLY SEX STUFF (2012) – Full Tr...,"Fuck, yeah! This is such a big night for you. ..."
326,https://scrapsfromtheloft.com/comedy/amy-schum...,AMY SCHUMER: THE LEATHER SPECIAL (2017) – Full...,"[announcer] Ladies and gentleman, and all you ..."
308,https://scrapsfromtheloft.com/comedy/anthony-j...,ANTHONY JESELNIK: CALIGULA (2013) – Full Trans...,Anthony Jeselnik. -MAN: Anthony. -WOMAN: Antho...
201,https://scrapsfromtheloft.com/comedy/ari-shaff...,ARI SHAFFIR: DOUBLE NEGATIVE (2017) – Full Tra...,Shaffir’s Double Negative collection includes ...
251,https://scrapsfromtheloft.com/comedy/aziz-ansa...,AZIZ ANSARI: BURIED ALIVE (2013) – Full Transc...,Ladies and gentlemen… Aziz Ansari! Thank you. ...
296,https://scrapsfromtheloft.com/comedy/aziz-ansa...,AZIZ ANSARI: INTIMATE MOMENTS FOR A SENSUAL EV...,"Recorded on July 1, 2009 at The Brentwood Thea..."


# Example of using tmdsimple

In [5]:
import tmdbsimple as tmdb
tmdb.API_KEY = 'bbc0a7ed538b8063d79dd9ab123aba20'

## Step 1: searching for the actor's id

In [17]:
name = 'ADEL KARAM'

In [18]:
search = tmdb.Search()
response = search.person(query=name)
print([s['id'] for s in search.results])

[53445]


## Step 2: using the id to search for info

In [12]:
identity = tmdb.People(53445)
response = identity.info()
print(identity.birthday, identity.gender, identity.place_of_birth)

1972-08-20 2 Beirut, Lebanon


handling multiple search results (usually the first one, but need to check)

In [15]:
response = search.person(query='ALI WONG')
[s['id'] for s in search.results]

[591835, 2946658, 3156949]

In [16]:
identity = tmdb.People(591835)
response = identity.info()
print(identity.birthday, identity.gender, identity.place_of_birth)

1982-04-19 1  San Francisco, California, USA


## Putting things together

In [59]:
def extraction(names):
    '''
    Taking in a list of actor/actress' names
    Returning a dictionary containing their birthday, gender and birthplace
    '''
    birthday = []
    gender = []
    birthplace = []
    
    search = tmdb.Search()
    for name in names:
        response = search.person(query=name)
        id_list = [s['id'] for s in search.results]
        # if there is no id result
        if len(id_list) < 1:
            print(f'No {name} found. Please check the name.')
            birthday.append(None)
            gender.append(None)
            birthplace.append(None)
        # if there is more than one result
        else:
            if len(id_list) > 1:
                print('Multiple people found.')
                print(id_list)
            first_id = id_list[0]
            identity = tmdb.People(first_id)
            response = identity.info()

            birthday.append(identity.birthday)
            gender.append(identity.gender)
            birthplace.append(identity.place_of_birth)
    return {'name': names, 'birthday': birthday, 'gender': gender, 'birthplace': birthplace}

In [65]:
new_dict = extraction(['AMY SCHUMER', 'goosdsds', 'dsfsfdsdfs', 'AZIZ ANSARI'])
pd.DataFrame.from_dict(new_dict)

No goosdsds found. Please check the name.
No dsfsfdsdfs found. Please check the name.


,name,birthday,gender,birthplace
0,AMY SCHUMER,1981-06-01,1.0,"New York, New York, USA"
1,goosdsds,None,NaN,None
2,dsfsfdsdfs,None,NaN,None
3,AZIZ ANSARI,1983-02-23,2.0,"Columbia, South Carolina, USA"
